In [1]:
# demonstration of calculating metrics for a neural network model using sklearn
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

# TensorFlow e tf.keras
import tensorflow as tf
from tensorflow import keras
import tensorflow_addons as tfa
import pandas as pd
# Bibliotecas Auxiliares
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

print(tf.__version__)

2023-03-05 16:39:59.230108: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-05 16:39:59.613435: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-05 16:40:00.580163: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/guilherme/cpu-tensorflow-marcelo/nvidia-smi/envs/tf/lib/
2023-03-05 16:40:00.582017: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7';

2.10.0


In [2]:
# configs
EPOCHS = 30
BATCH_SIZE = 32

baseFolder = "../data_2019_processed/"

In [3]:
# y_test     = Array with real values
# yhat_probs = Array with predicted values
def printMetrics(y_test,yhat_probs):
    # predict crisp classes for test set deprecated
    #yhat_classes = model.predict_classes(X_test, verbose=0)
    #yhat_classes = np.argmax(yhat_probs,axis=1)
    yhat_classes = yhat_probs.round()
    # accuracy: (tp + tn) / (p + n)
    accuracy = accuracy_score(y_test, yhat_classes)
    print('Accuracy: %f' % accuracy)
    # precision tp / (tp + fp)
    precision = precision_score(y_test, yhat_classes)
    print('Precision: %f' % precision)
    # recall: tp / (tp + fn)
    recall = recall_score(y_test, yhat_classes)
    print('Recall: %f' % recall)
    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(y_test, yhat_classes)
    print('F1 score: %f' % f1)
    # kappa
    kappa = cohen_kappa_score(y_test, yhat_classes)
    print('Cohens kappa: %f' % kappa)
    # ROC AUC
    auc = roc_auc_score(y_test, yhat_probs)
    print('ROC AUC: %f' % auc)
    # confusion matrix
    print("\Confusion Matrix")
    matrix = confusion_matrix(y_test, yhat_classes)
    print(matrix)
    
    results = dict()
    results['accuracy'] = accuracy
    results['precision'] = precision
    results['recall'] = recall
    results['f1_score'] = f1
    results['cohen_kappa_score'] = kappa
    results['roc_auc_score'] = auc
    return results

def showGlobalMetrics(metrics):
    accuracy,precision,recall,f1_score,cohen_kappa_score,roc_auc_score = 0,0,0,0,0,0
    for metric in metrics:
        accuracy = accuracy + metric['accuracy']
        precision = precision + metric['precision']
        recall = recall + metric['recall']
        f1_score = f1_score + metric['f1_score']
        cohen_kappa_score = cohen_kappa_score + metric['cohen_kappa_score']
        roc_auc_score = roc_auc_score + metric['roc_auc_score']
        
    # mean
    size = len(metrics)
    print(size)
    accuracy = accuracy / size
    precision = precision / size
    recall = recall / size
    f1_score = f1_score / size
    cohen_kappa_score = cohen_kappa_score / size
    roc_auc_score = roc_auc_score / size
    
    #show:\
    print("accuracy: ",accuracy)
    print("precision: ",precision)
    print("recall: ",recall)
    print("f1_score: ",f1_score)
    print("cohen_kappa_score: ",cohen_kappa_score)
    print("roc_auc_score: ",roc_auc_score)
    
def transform_data_type(dataframe):
    
    # transform inputs
    for column in inputFeatures:
        dataframe[column] = dataframe[column].astype('float32')
    
    # transform outputs
    for column in outputClasses:
        dataframe[column] = dataframe[column].astype('float32')
    
    return dataframe

# one-hot encoding function
def transform_output_nominal_class_into_one_hot_encoding(dataset):
    # create two classes based on the single class
    one_hot_encoded_data = pd.get_dummies(dataset['class'])
    #print(one_hot_encoded_data)
    dataset['awake'] = one_hot_encoded_data['awake']
    dataset['asleep'] = one_hot_encoded_data['asleep']
    
    return dataset

# one-hot encoding function
def transform_output_numerical_class_into_one_hot_encoding(dataset):
    # create two classes based on the single class
    one_hot_encoded_data = pd.get_dummies(dataset['class'])
    #print(one_hot_encoded_data)
    dataset['awake'] = one_hot_encoded_data[0]
    dataset['asleep'] = one_hot_encoded_data[1]
    
    return dataset

In [4]:
X_train = pd.read_csv(baseFolder+"train/allData-classification-numeric-normalized.csv")
X_test  = pd.read_csv(baseFolder+"test/allData-classification-numeric-normalized.csv")

In [5]:
print(X_train.info())
X_train

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 407451 entries, 0 to 407450
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   activity            407451 non-null  float64
 1   location            407451 non-null  float64
 2   timestamp           407451 non-null  float64
 3   day_of_week         407451 non-null  float64
 4   light               407451 non-null  float64
 5   phone_lock          407451 non-null  float64
 6   proximity           407451 non-null  float64
 7   sound               407451 non-null  float64
 8   time_to_next_alarm  407451 non-null  float64
 9   minutes_day         407451 non-null  float64
 10  timestamp_text      407451 non-null  object 
 11  class               407451 non-null  object 
dtypes: float64(10), object(2)
memory usage: 37.3+ MB
None


,activity,location,timestamp,day_of_week,light,phone_lock,proximity,sound,time_to_next_alarm,minutes_day,timestamp_text,class
0,0.00,0.0,0.000869,0.0,0.000175,0.0,1.0,0.000000,0.982143,0.597637,2018-05-15 14:20:45,awake
1,0.00,0.0,0.000869,0.0,0.000175,0.0,1.0,0.000000,0.982143,0.597637,2018-05-15 14:20:45,awake
2,0.25,0.5,0.000869,0.0,0.000165,0.0,1.0,0.604408,0.982044,0.598332,2018-05-15 14:21:15,awake
3,0.25,0.5,0.000869,0.0,0.001449,0.0,1.0,0.604408,0.982044,0.598332,2018-05-15 14:21:45,awake
4,0.25,0.5,0.000869,0.0,0.000198,0.0,1.0,0.601849,0.981944,0.599027,2018-05-15 14:22:15,awake
...,...,...,...,...,...,...,...,...,...,...,...,...
407446,0.25,1.0,0.006924,0.0,0.000000,1.0,1.0,0.644370,0.992956,0.549687,2018-06-12 13:11:39,awake
407447,0.25,1.0,0.006924,0.0,0.000000,1.0,1.0,0.644370,0.992956,0.550382,2018-06-12 13:12:09,awake
407448,0.25,1.0,0.006924,0.0,0.000538,1.0,1.0,0.624127,0.992857,0.551077,2018-06-12 13:13:37,awake
407449,0.00,1.0,0.006924,0.0,0.000000,0.0,0.0,0.540295,0.992758,0.551772,2018-06-12 13:14:07,awake


In [6]:
# create two classes based on the single class

# transform output to one_hot_encoding for the testing dataset
X_test = transform_output_nominal_class_into_one_hot_encoding(X_test)

# transform output to one_hot_encoding for the testing dataset
X_train = transform_output_nominal_class_into_one_hot_encoding(X_train)

In [7]:
# activity	location	timestamp	day_of_week	light	phone_lock	proximity	sound	time_to_next_alarm	minutes_day
inputFeatures = ["activity","location","day_of_week","light","phone_lock","proximity","sound","time_to_next_alarm", "minutes_day"]
#inputFeatures = ["activity","location","day_of_week","light","phone_lock","proximity","sound", "minutes_day"]

outputClasses = ["awake","asleep"]



def create_keras_model(inputFeatures,outputClasses):
    return tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=(len(inputFeatures),)),
        tf.keras.layers.Dense(len(inputFeatures)),
        tf.keras.layers.Dense(14, activation=tf.keras.activations.relu),
        tf.keras.layers.Dense(7, activation=tf.keras.activations.relu),
        tf.keras.layers.Dense(len(outputClasses), activation='softmax')
    ])

In [8]:
print("--------------")
print("--------------")
print("Initializing raw test")
print("--------------")
print("Batch Size: ",BATCH_SIZE)
print("Epochs: ",EPOCHS)
print("Input/Features:",len(inputFeatures))
print("Outputs:",len(outputClasses))
print("--------------")

# selects the data to train and test
X_train_data = pd.DataFrame(data=X_train,columns=inputFeatures)
y_train_data = pd.DataFrame(data=X_train,columns=outputClasses)
# selec test dataset (fixed to all)
X_test_data = pd.DataFrame(data=X_test,columns=inputFeatures)
y_test_data = pd.DataFrame(data=X_test,columns=outputClasses)

# instanciates the model
model = create_keras_model(inputFeatures,outputClasses)

# compiles the model
model.compile(optimizer='adam',
             # loss='binary_crossentropy',loss='categorical_crossentropy',
              #loss='binary_crossentropy',  sparse_categorical_crossentropy         
              loss='categorical_crossentropy',
              metrics=[tf.keras.metrics.CategoricalAccuracy()])
# 
model.fit(X_train_data, y_train_data, epochs=EPOCHS,batch_size=BATCH_SIZE)

results = model.evaluate(X_test_data, y_test_data, return_dict=True)

print(results)

--------------
--------------
Initializing raw test
--------------
Batch Size:  32
Epochs:  30
Input/Features: 9
Outputs: 2
--------------


2023-03-05 16:40:05.537311: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-05 16:40:05.628097: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-05 16:40:05.628425: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-05 16:40:05.630200: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

Epoch 1/30
   36/12733 [..............................] - ETA: 55s - loss: 0.6908 - categorical_accuracy: 0.5564  

2023-03-05 16:40:08.961745: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


12733/12733 [==============================] - 72s 6ms/step - loss: 0.5453 - categorical_accuracy: 0.7412
Epoch 2/30
12733/12733 [==============================] - 70s 5ms/step - loss: 0.5181 - categorical_accuracy: 0.7626
Epoch 3/30
12733/12733 [==============================] - 72s 6ms/step - loss: 0.5125 - categorical_accuracy: 0.7667
Epoch 4/30
12733/12733 [==============================] - 74s 6ms/step - loss: 0.5092 - categorical_accuracy: 0.7686
Epoch 5/30
12733/12733 [==============================] - 72s 6ms/step - loss: 0.5076 - categorical_accuracy: 0.7687
Epoch 6/30
12733/12733 [==============================] - 72s 6ms/step - loss: 0.5067 - categorical_accuracy: 0.7684
Epoch 7/30
12733/12733 [==============================] - 74s 6ms/step - loss: 0.5060 - categorical_accuracy: 0.7690
Epoch 8/30
12733/12733 [==============================] - 73s 6ms/step - loss: 0.5054 - categorical_accuracy: 0.7693
Epoch 9/30
12733/12733 [==============================] - 80s 6ms/step - lo

In [9]:
yhat_probs = model.predict(X_test_data)
# dataset
probss = pd.DataFrame(data=yhat_probs,columns=['awake','asleep'])

test = list()
print('')
print('awake')
res = printMetrics(y_test_data['awake'],probss['awake'])
test.append(res)
print('')
print('asleep')
res = printMetrics(y_test_data['asleep'],probss['asleep'])
test.append(res)
print('')
print('Global')
showGlobalMetrics(test)

4259/4259 [==============================] - 5s 1ms/step

awake
Accuracy: 0.746210
Precision: 0.781045
Recall: 0.887324
F1 score: 0.830799
Cohens kappa: 0.331424
ROC AUC: 0.702567
\Confusion Matrix
[[16782 23805]
 [10783 84916]]

asleep
Accuracy: 0.746210
Precision: 0.608816
Recall: 0.413482
F1 score: 0.492487
Cohens kappa: 0.331424
ROC AUC: 0.702567
\Confusion Matrix
[[84916 10783]
 [23805 16782]]

Global
2
accuracy:  0.7462101756600091
precision:  0.6949302936141362
recall:  0.6504029729718122
f1_score:  0.6616433579255422
cohen_kappa_score:  0.33142445713277224
roc_auc_score:  0.7025669691183594
